<a href="https://colab.research.google.com/github/priyankayadav021/Leaf-Constraint-Minimum-Spanning-Tree/blob/main/k_means_Based_Heuristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx
from math import sqrt
%matplotlib inline
from typing import Sized
import sys # Library for INT_MAX
import timeit

In [ ]:
def dis_euclidean(x1,x2,y1,y2):
  dist = sqrt((x1-x2)**2+(y1-y2)**2)
  return dist

In [ ]:
f = open('file_part1','r')
n = int(f.readline())
k = int(.1*n)
x = []
y = []
lines = f.readlines()
for line in lines:
    a = line.split()
    x.append(float(a[0]))
    y.append(float(a[1]))

graph = [[0 for x in range(n)] for y in range(n)] 
for i in range(0,n):
  for j in range(0,n):
    #print(i,j)
    graph[i][j] = dis_euclidean(x[i],x[j],y[i],y[j])

#print(graph)

df = pd.DataFrame({
    'x': x,
    'y': y
    })
#df.head()

In [ ]:
def kmeans():
  centre=[0 for x in range(k)]
  km = KMeans(k)
  y_predicted = km.fit_predict(df[['x','y']])
  df['cluster'] = y_predicted
  df['node'] = np.arange(len(df))
  #print(df.head())
  centres = pd.DataFrame(km.cluster_centers_ , columns = ['x','y'])
  #print(centres.head())
  count = 0
  c = 0
  
  while(count<k):
    mini = 99.0
    for i in range(len(df.node)):
      if( count == df.cluster[i]):
        dis = dis_euclidean(df.x[i], centres.x[count], df.y[i], centres.y[count])      
        #dis = round(dis,3)
        if(float(mini) > float(dis)):
          mini = dis
          c = df.node[i]
  
    centre[count]= c
    count = count + 1
  #print(centre)
  return centre

  

In [ ]:
def minKey(key,mstSet):
		min = sys.float_info.max
		min_index=0
		for v in range(n):
			if (key[v] < min and mstSet[v] == False):
				min = key[v]
				min_index = v
		return min_index

	
def primMST(x):
		tree = [[0 for x in range(n)] for y in range(n)]
		mstSet = [True]*n
		flag = [0]*n
		key = [sys.float_info.max]*n
		for a in range(0, k):
			mstSet[centre[a]] = False			
		parent = [None] * n
		key[x] = 0.0
		parent[x] = -1	
		for cout in range(n):
			u = minKey(key,mstSet)
			mstSet[u] = True
		
			for v in range(n):
				if(graph[u][v] > 0 and mstSet[v] == False and key[v] > graph[u][v]):
					parent[v] = u
					flag[v] = 1
					key[v] = graph[u][v]

		weight = 0.0
		for i in range(n):
				if(flag[i]):
					tree[parent[i]][i] = graph[parent[i]][i]
					tree[i][parent[i]] = graph[i][parent[i]]
		for i in range(n):
				for j in range(n):
					 if(i<=j):
						 weight+= tree[i][j]
		#tree_plot(tree)
		return tree
		
		
def make_tree(tree):	
		count=0
		while(count<k):
			for i in range(len(df.node)):
				if( count == df.cluster[i]):
					tree[centre[count]][df.node[i]] = graph[centre[count]][df.node[i]]
					tree[df.node[i]][centre[count]] = graph[df.node[i]][centre[count]]
			count = count + 1

		weight = 0.0
		for i in range(n):
			for j in range(n):
				if(i<=j):
					weight+= tree[i][j]

		return weight

In [ ]:
def tree_plot(tree):
  l=[]
  #file=open("b.txt")
  #arr = np.genfromtxt(file,delimiter=' ',dtype=None)
  arr = np.array(tree)
  #print(arr)
  for i in range(0,n):
      for j in range(0,n):
          if(arr[i][j] != 0.):
              l.append((i,j))
  #print(l)
  plt.figure(figsize = (10, 6))
  G=nx.DiGraph()
  G.add_edges_from(l)
  pos=nx.spring_layout(G)
  nx.draw_networkx_nodes(G,pos,node_size=205)
  nx.draw_networkx_edges(G,pos,edgelist=G.edges(),edge_color='red')
  nx.draw_networkx_labels(G,pos)
  plt.show()

In [ ]:
weight=0
temp=99.0
time=0
for i in range(20): 
  start = timeit.default_timer()
  centre=kmeans()
  #print(centre)
  tree=primMST(centre[0])
  weight = make_tree(tree)
  #print(weight)
  stop = timeit.default_timer()
  if(temp>weight):
    temp=weight
    time=stop - start
  #tree_plot(tree)
  #stop = timeit.default_timer()
print('Time: ', time)  
print("weight",temp)
 

Time:  0.2709585610000431
weight 20.501420549192293
